In [3]:
from esrd.hcc_esrd import Diagnosis, Beneficiary, ICDType, score,regvars, EntitlementReason
from pyDatalog import pyDatalog
import pandas as pd
import random

In [4]:
#reading files
diag_f= "/home/user/Documents/projects/hcc/zip files/validation_data/ESRD/DIAG.csv"
person_f= "/home/user/Documents/projects/hcc/zip files/validation_data/ESRD/PERSON.csv"
output_f= "/home/user/Documents/projects/hcc/zip files/validation_data/ESRD/output.csv"
hcc_coeff_f= "esrd/hcccoefn_cleaned.csv"

df_diag= pd.read_csv(diag_f)
df_person= pd.read_csv(person_f)
df_output= pd.read_csv(output_f)
hcc_coeff= pd.read_csv(hcc_coeff_f, names= ['coeff', 'value'])

In [5]:
raf_type= {'DI': 'valid_dialysis_variables', 'DNE': 'valid_dialysis_new_enrollee_variables',
                        'GC': 'valid_functioning_graft_community_variables', 'GI': 'valid_functioning_graft_institutional_variables',
                        'GNE': 'valid_functioning_graft_new_enrolle_regression_variables'
                        }

raf_type

{'DI': 'valid_dialysis_variables',
 'DNE': 'valid_dialysis_new_enrollee_variables',
 'GC': 'valid_functioning_graft_community_variables',
 'GI': 'valid_functioning_graft_institutional_variables',
 'GNE': 'valid_functioning_graft_new_enrolle_regression_variables'}

In [7]:
df_diag.head()


,HICNO,Dx
0,001148575Z,R55
1,001148575Z,R079
2,001148575Z,Z0000
3,001148575Z,Z09
4,001148575Z,H353221


In [8]:
df_person.head()


,HICNO,SEX,DOB,LTIMCAID,NEMCAID,OREC
0,{0014370513,2,8/7/1938,1,0,1.0
1,{0037415816,2,6/10/1916,0,0,0.0
2,{0037481943,1,7/20/1937,0,0,1.0
3,{0037481943,1,7/20/1937,1,0,1.0
4,{0039989643,1,5/21/1961,1,0,1.0


In [9]:
df_output.head()

,HICNO,SEX,DOB,NEMCAID,OREC,DISABL,ORIGDS,MCAID,DF_DG,DF_POSTG,...,SCORE_GRAFT_COMM_DUR10PL_GE65,SCORE_GRAFT_COMM_DUR10PL_LT65,SCORE_GRAFT_INST_DUR4_9_GE65,SCORE_GRAFT_INST_DUR4_9_LT65,SCORE_GRAFT_INST_DUR10PL_GE65,SCORE_GRAFT_INST_DUR10PL_LT65,SCORE_GRAFT_NE_DUR4_9_GE65,SCORE_GRAFT_NE_DUR4_9_LT65,SCORE_GRAFT_NE_DUR10PL_GE65,SCORE_GRAFT_NE_DUR10PL_LT65
0,001148575A,2,12/24/21,0,0,0,0,NaN,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,3.876,1.241,2.509,1.241
1,001201070A,1,6/2/27,0,0,0,0,NaN,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,4.249,1.614,2.882,1.614
2,001201070B,2,7/25/27,0,0,0,0,NaN,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,4.000,1.365,2.633,1.365
3,001208085A,2,6/15/23,0,0,0,0,NaN,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,4.000,1.365,2.633,1.365
4,001208292A,2,4/20/27,0,0,0,0,NaN,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,4.000,1.365,2.633,1.365


In [10]:
print('selecting random row from the person df')
#print(df_person.index)
id=random.randint(df_person.index[0],df_person.index[-1] )
temp_patient= dict(df_person.loc[id])
print(temp_patient)

selecting random row from the person df
{'HICNO': '153400235A', 'SEX': 1, 'DOB': '12/22/1949', 'LTIMCAID': 0, 'NEMCAID': 0, 'OREC': 0.0}


In [11]:
print("getting diagnosis data for {}: {}".format('Patient_ID', temp_patient['Patient_ID'] ))
temp_diag= df_diag.query("Patient_ID=='{}'".format(temp_patient['Patient_ID']))
temp_diag

KeyError: 'Patient_ID'

In [11]:
from hcc import Diagnosis, Beneficiary, ICDType, score,regvars, EntitlementReason
from pyDatalog import pyDatalog

sex= {'f':'female', 'm': 'male'}

temp_sex= sex[temp_patient['Gender'].lower()]

temp_dob= ''.join(temp_patient['Patient_DOB'].split('-'))
temp_age_upto= ''.join(temp_patient['Month_of_eligibility'].split('-'))

char1, char2, char3=temp_patient['RAF_Type']



if char1 =='E' and char3 =='D':
    orec= 3
else:
    orec= {'A':0, 'D':1}[char3]
    
print(temp_sex, temp_dob,temp_age_upto, orec)

person= Beneficiary(hicno= temp_patient['Patient_ID'], sex= temp_sex ,dob= temp_dob, age_upto= temp_age_upto , original_reason_entitlement= orec, medicaid=True, )
print(person)

female 19341006 20170101 0
ID:1848062742999,DOB:1934-10-06 00:00:00,age_upto:2017-01-01 00:00:00


In [12]:
for code in temp_diag[' ICD10'].values:
    person.add_diagnosis(Diagnosis(person,code,ICDType.TEN))

In [13]:
pyDatalog.create_terms("Vars")

temp_raf_type= raf_type[temp_patient['RAF_Type']]

conditiion_categories= regvars(person, temp_raf_type, Vars)[0][0].split(',')

print("conditiion_categories: {} for RAF_Type: {}".format(conditiion_categories,temp_patient['RAF_Type']))

conditiion_categories: ['F80_84', 'HCC107', 'HCC19', 'HCC21', 'HCC8', 'HCC84', 'HCC85', 'HCC85_gDiabetesMellit'] for RAF_Type: CNA


In [14]:
func1= lambda c : '_'.join([temp_patient['RAF_Type'],c]).lower()
func2= lambda x: round(list(hcc_coeff.query("coeff=='{}'".format(x))['value'])[0],3)

[ {func1(c):func2(func1(c))} for c in conditiion_categories]

[{'cna_f80_84': 0.537},
 {'cna_hcc107': 0.4},
 {'cna_hcc19': 0.104},
 {'cna_hcc21': 0.545},
 {'cna_hcc8': 2.625},
 {'cna_hcc84': 0.302},
 {'cna_hcc85': 0.323},
 {'cna_hcc85_gdiabetesmellit': 0.154}]

In [13]:
set(df_output.query("Patient_ID=='{}'".format(temp_patient['Patient_ID']))['Observation'])

{'CNA_F80_84', 'CNA_HCC85'}

In [14]:
df_output.query("Patient_ID=='{}'".format(temp_patient['Patient_ID']))

,Run_date,year_of_eligibility,Patient_ID,DOB,Gender,Group_code,Plan_type,Observation,RAF coefficient,RAF_type
915,2018-09-24,2017-05-01,1286939515999,1933-09-18,F,NYMCRWP0,MEDICARE ADVANTAGE,CNA_F80_84,0.537,CNA
916,2018-09-24,2017-06-01,1286939515999,1933-09-18,F,NYMCRWP0,MEDICARE ADVANTAGE,CNA_F80_84,0.537,CNA
917,2018-09-24,2017-07-01,1286939515999,1933-09-18,F,NYMCRWP0,MEDICARE ADVANTAGE,CNA_F80_84,0.537,CNA
918,2018-09-24,2017-08-01,1286939515999,1933-09-18,F,NYMCRWP0,MEDICARE ADVANTAGE,CNA_F80_84,0.537,CNA
919,2018-09-24,2017-09-01,1286939515999,1933-09-18,F,NYMCRWP0,MEDICARE ADVANTAGE,CNA_F80_84,0.537,CNA
920,2018-09-24,2017-10-01,1286939515999,1933-09-18,F,NYMCRWP0,MEDICARE ADVANTAGE,CNA_F80_84,0.537,CNA
921,2018-09-24,2017-11-01,1286939515999,1933-09-18,F,NYMCRWP0,MEDICARE ADVANTAGE,CNA_F80_84,0.537,CNA
922,2018-09-24,2017-12-01,1286939515999,1933-09-18,F,NYMCRWP0,MEDICARE ADVANTAGE,CNA_F80_84,0.537,CNA
923,2018-09-24,2018-01-01,1286939515999,1933-09-18,F,NYMCRWP0,MEDICARE ADVANTAGE,CNA_F80_84,0.537,CNA
924,2018-09-24,2018-01-01,1286939515999,1933-09-18,F,NYMCRWP0,MEDICARE ADVANTAGE,CNA_HCC85,0.323,CNA


In [29]:
list(temp_diag[' ICD10'])

['E119', 'E119', 'E119', 'E119', 'E119', 'E119', 'E119']

In [6]:
import os

os.stat_result(st_mode=33188, st_ino=42074268, st_dev=2049, st_nlink=1, st_uid=1000, st_gid=1000, st_size=61894639, st_atime=1545155267, st_mtime=1530284193, st_ctime=1544552358)